# RSNA Intracranial Hemorrhage Detection 

<b>What am i predicting?</b><br/><br/>
In this competition our goal is to predict intracranial hemorrhage and its subtypes. Given an image the we need to predict probablity of each subtype. This indicates its a multilabel classification problem.

<b>Competition Evaluation Metric</b><br/><br/>
Evaluation metric is weighted multi-label logarithmic loss. So for given image we need to predict probality for each subtype. There is also an any label, which indicates that a hemorrhage of ANY kind exists in the image. The any label is weighted more highly than specific hemorrhage sub-types.

<b>Note:</b>The weights for each subtype for calculating weighted multi-label logarithmic loss is **not** given as part of the competition. We will be using binary cross entropy loss as weights are not available

<b>Dataset Description</b>

The dataset is divided into two parts

1. Train
2. Test

**1. Train**
Number of rows: 40,45,548 records.
Number of columns: 2

Columns:

**Id**: An image Id. Each Id corresponds to a unique image, and will contain an underscore.

Example: ID_28fbab7eb_epidural. So the Id consists of two parts one is image file id ID_28fbab7eb and the other is sub type name

**Label**: The target label whether that sub-type of hemorrhage (or any hemorrhage in the case of any) exists in the indicated image. 1 --> Exists and 0 --> Doesn't exist.

**2. Test**
Number of rows: 4,71,270 records.

Columns:

**Id**: An image Id. Each Id corresponds to a unique image, and will contain an underscore.

Example: ID_28fbab7eb_epidural. So the Id consists of two parts one is image file id ID_28fbab7eb and the other is sub type name

In [ ]:
import numpy as np
import pandas as pd
import pydicom
import os
import glob
import random
import cv2
import tensorflow as tf
from math import ceil, floor
from tqdm import tqdm
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
from math import ceil, floor
import keras
import keras.backend as K
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model, load_model
from keras.applications.resnet50 import ResNet50
from keras.utils import Sequence
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# Random Seed
SEED = 42
np.random.seed(SEED)

# some constants
TEST_SIZE = 0.06
HEIGHT = 224
WIDTH = 224
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16

# Train and Test folders
input_folder = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/'
path_train_img = input_folder + 'stage_2_train/'
path_test_img = input_folder + 'stage_2_test/'

In [ ]:
train_df = pd.read_csv(input_folder + 'stage_2_train.csv')
train_df.head()

In [ ]:
# extract subtype
train_df['sub_type'] = train_df['ID'].apply(lambda x: x.split('_')[-1])
# extract filename
train_df['file_name'] = train_df['ID'].apply(lambda x: '_'.join(x.split('_')[:2]) + '.dcm')
train_df.head()

In [ ]:
train_df.shape

In [ ]:
# remove duplicates
train_df.drop_duplicates(['Label', 'sub_type', 'file_name'], inplace=True)
train_df.shape

In [ ]:
print("Number of train images availabe:", len(os.listdir(path_train_img)))

In [ ]:
train_final_df = pd.pivot_table(train_df.drop(columns='ID'), index="file_name", \
                                columns="sub_type", values="Label")
train_final_df.head()

In [ ]:
train_final_df.shape

In [ ]:
# Invalid image ID_6431af929.dcm
train_final_df.drop('ID_6431af929.dcm', inplace=True)

In [ ]:
train_small_df = train_final_df.head(0)

epidural_df = train_final_df[train_final_df.epidural == 1]
intraparenchymal_df = train_final_df[train_final_df.intraparenchymal == 1]
intraventricular_df = train_final_df[train_final_df.intraventricular == 1]
subarachnoid_df = train_final_df[train_final_df.subarachnoid == 1]
subdural_df = train_final_df[train_final_df.subdural == 1]

non_df=train_final_df[(train_final_df.epidural == 0) & (train_final_df.intraparenchymal == 0) & \
                      (train_final_df.intraventricular == 0) & (train_final_df.subarachnoid == 0) & \
                      (train_final_df.subdural == 0)]

train_small_df = pd.concat([train_small_df,epidural_df[:2000],intraparenchymal_df[:2000],\
                            intraventricular_df[:2000],subarachnoid_df[:2000],subdural_df[:2000],\
                            non_df[:10000] ])

In [ ]:
def get_dicom_field_value(val):
    if type(val) == pydicom.multival.MultiValue:
        return int(val[0])
    else:
        return int(val)

def get_windowing(data):
    dicom_fields = [data.WindowCenter, data.WindowWidth, \
                    data.RescaleSlope, data.RescaleIntercept]
    return [get_dicom_field_value(x) for x in dicom_fields]

def get_windowed_image(image, wc, ww, slope, intercept):
    img = (image*slope +intercept)
    img_min = wc - ww//2
    img_max = wc + ww//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    return img 


def _normalize(img):
    if img.max() == img.min():
        return np.zeros(img.shape)
    return 2 * (img - img.min())/(img.max() - img.min()) - 1

def _read(path, desired_size=(224, 224)):
    # 1. read dicom file
    dcm = pydicom.dcmread(path)
    
    # 2. Extract meta data features
    # window center, window width, slope, intercept
    window_params = get_windowing(dcm)

    try:
        # 3. Generate windowed image
        img = get_windowed_image(dcm.pixel_array, *window_params)
    except:
        img = np.zeros(desired_size)

    img = _normalize(img)

    if desired_size != (512, 512):
        # resize image
        img = cv2.resize(img, desired_size, interpolation = cv2.INTER_LINEAR)
    return img[:,:,np.newaxis]

In [ ]:
_read(path_train_img + 'ID_ffff922b9.dcm', (128, 128)).shape

In [ ]:
plt.imshow(
    _read(path_train_img + 'ID_ffff922b9.dcm', (128, 128))[:, :, 0]
)

In [ ]:
# Train Data Generator
class TrainDataGenerator(keras.utils.Sequence):

    def __init__(self, dataset, labels, batch_size=16, img_size=(512, 512), img_dir = path_train_img, *args, **kwargs):
        self.dataset = dataset
        self.ids = dataset.index
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X, Y = self.__data_generation(indices)
        return X, Y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))
        np.random.shuffle(self.indices)
        
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size, 3))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
            image = _read(self.img_dir + ID, self.img_size)
            X[i,] = image            
            Y[i,] = self.labels.iloc[index].values        
        return X, Y
    
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, ids, labels, batch_size = 5, img_size = (512, 512), img_dir = path_test_img, \
                 *args, **kwargs):
        self.ids = ids
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.ids[k] for k in indices]
        X = self.__data_generation(list_IDs_temp)
        return X

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 3))
        for i, ID in enumerate(list_IDs_temp):
            image = _read(self.img_dir + ID, self.img_size)
            X[i,] = image            
        return X

As we have seen in EDA notebook that we have very few epidural subtypes so we need oversample this sub type

In [ ]:
# Oversampling
#epidural_df = train_final_df[train_final_df.epidural == 1]
#train_final_df = pd.concat([train_final_df, epidural_df])
#print('Train Shape: {}'.format(train_final_df.shape))

In [ ]:
# load test set
test_df = pd.read_csv(input_folder + 'stage_2_sample_submission.csv')
test_df.head()

In [ ]:
# extract subtype
test_df['sub_type'] = test_df['ID'].apply(lambda x: x.split('_')[-1])
# extract filename
test_df['file_name'] = test_df['ID'].apply(lambda x: '_'.join(x.split('_')[:2]) + '.dcm')

test_df = pd.pivot_table(test_df.drop(columns='ID'), index="file_name", \
                                columns="sub_type", values="Label")
test_df.head()

test_df.shape

In [ ]:
test_df.head()

In [ ]:
base_model =  ResNet50(weights = 'imagenet', include_top = False, \
                                 pooling = 'avg', input_shape = (HEIGHT, WIDTH, 3))
x = base_model.output
#x = Dropout(0.125)(x)
output_layer = Dense(6, activation = 'sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer = Adam(learning_rate = 0.0001), 
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])

In [ ]:
# https://github.com/trent-b/iterative-stratification
# Mutlilabel stratification
splits = StratifiedShuffleSplit(n_splits = 2, test_size = TEST_SIZE, random_state = SEED)
file_names = train_small_df.index
labels = train_small_df.values
# Lets take only the first split
split = next(splits.split(file_names, labels))
train_idx = split[0]
valid_idx = split[1]
submission_predictions = []
len(train_idx), len(valid_idx)


In [ ]:
# train data generator
data_generator_train = TrainDataGenerator(train_small_df.iloc[train_idx], 
                                                train_small_df.iloc[train_idx], 
                                                TRAIN_BATCH_SIZE, 
                                                (WIDTH, HEIGHT))

# validation data generator
data_generator_val = TrainDataGenerator(train_small_df.iloc[valid_idx], 
                                            train_small_df.iloc[valid_idx], 
                                            VALID_BATCH_SIZE, 
                                            (WIDTH, HEIGHT))

In [ ]:
len(data_generator_train), len(data_generator_val)

Competition evaluation metric is evaluated based on weighted log loss but we haven't given weights for each subtype but as per discussion from this thread https://www.kaggle.com/c/rsna-intracranial-hemorrhage-detection/discussion/109526#latest-630190 any has a wieght of 2 than other types below sample is taken from the discussion threas

For a single epoch we are going to train only last 5 layers of Efficient. Since we have a large number of images around 600k so its better to train the all the layers on the whole train dataset but due its high computation resources required to train we only goin to train last five layers on whole dataset and for rest of epochs we only train on a sample of dataset but will train all the layers.

In [ ]:
model.fit_generator(generator = data_generator_train,
                            validation_data = data_generator_val,
                            epochs = 5,
                            verbose = 1)

In [ ]:
model.save('model.h5')

In [ ]:
#!pip install gdown

In [ ]:
#!gdown https://drive.google.com/uc?id=1kZmMCCBOWSjCZjz2XWaouDIj5gFn2D-q

In [ ]:
#!cp "model (4).h5" model.h5

In [ ]:
X = np.empty((1, 224, 224, 3))
X[0, ] = _read(path_train_img + 'ID_000012eaf.dcm', (224, 224))
plt.imshow(
    _read(path_train_img + 'ID_000012eaf.dcm', (224, 224))[:, :, 0]
 )

In [ ]:
# np.argmax(model.predict(X))

y = model.predict(X)
pred = zip(y.tolist()[0], train_small_df.columns)
for i in pred:
    print(i)

In [ ]:
from tqdm import tqdm

In [ ]:
cols = list(train_small_df.columns)

In [ ]:
# We have preditions for each of the image
# We need to make 6 rows for each of file according to the subtype
ids = []
values = []
for i, j in tqdm(zip(pred, test_df.index.to_list())):
#     print(i, j)
    # i=[any_prob, epidural_prob, intraparenchymal_prob, intraventricular_prob, subarachnoid_prob, subdural_prob]
    # j = filename ==> ID_xyz.dcm
    for k in range(i.shape[0]):
        ids.append([j.replace('.dcm', '_' + cols[k])])
        values.append(i[k])      

In [ ]:
df = pd.DataFrame(data=ids)
df.head()

In [ ]:
sample_df = pd.read_csv(input_folder + 'stage_2_sample_submission.csv')
sample_df.head()

In [ ]:
df['Label'] = values
df.columns = sample_df.columns
df.head()

In [ ]:
df.to_csv('submission.csv', index=False)

In [ ]:
create_download_link(filename='submission.csv')